In [1]:
from scipy.io import loadmat
import numpy as np
import torch 
import torch.nn as nn
import torch.optim as optim
from torchsummary import summary
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import wandb
from torch.optim.lr_scheduler import ReduceLROnPlateau

### Gettig Y in the correct Format !

In [ ]:
Y_mat_location = r'/home/mnl/Desktop/University/Fall 2024-2025/VIPP/CODES/Python-MATLAB/MiMo/Matlab_Data/Y_1.mat'
r_q_all_location = r'/home/mnl/Desktop/University/Fall 2024-2025/VIPP/CODES/Python-MATLAB/MiMo/Matlab_Data/r_q_all_1.mat'
Y = loadmat(Y_mat_location)  
Y = Y['Y']
Y = np.transpose(Y,(2,0,1))
dimension = 10000
# now the shape is 50000, 16, 20 , I want to make it 50000 x 16, 20
Y = np.reshape(Y,(dimension*16,20))

Y_real = np.real(Y)
Y_imag = np.imag(Y)
# make the shape from 800000 x 20 to 800000 x 40
Y = np.concatenate((Y_real,Y_imag),axis=1)
print(f"the shape of Y = {Y.shape}")

r_q_all = loadmat(r_q_all_location)
r_q_all = r_q_all['r_q_all']

r_q_all = np.transpose(r_q_all,(1,0))
r_q_all = np.reshape(r_q_all,dimension* 16)
print(f"the shape of r_q_all = {r_q_all.shape}")

the shape of Y = (160000, 40)
the shape of r_q_all = (160000,)


### The model 

In [ ]:
class Complex_Model(nn.Module):
    '''Input is expected to be of shape [batch_size,n_inputs], where n_inputs = n_pilots*2'''
    def __init__(self, n_inputs = 40):
        
        super(Complex_Model, self).__init__()
        self.n_inputs = n_inputs
        '''Common branch after concatenation'''
        self.common_branch = nn.Sequential(
            nn.Linear(self.n_inputs, 256),
            #nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Linear(256, 256),
            #nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Linear(256, 128),
            #nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Linear(128, 16),
            # nn.BatchNorm1d(16),
            nn.ReLU(),
            nn.Linear(16, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.common_branch(x)
        return x
'''Small Test for the model'''
model = Complex_Model()
# Simulate a single tensor input with shape [batch_size, 8, 8, 2]
summary(model, input_size=(40,))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                  [-1, 256]          10,496
              ReLU-2                  [-1, 256]               0
            Linear-3                  [-1, 256]          65,792
              ReLU-4                  [-1, 256]               0
            Linear-5                  [-1, 128]          32,896
              ReLU-6                  [-1, 128]               0
            Linear-7                   [-1, 16]           2,064
              ReLU-8                   [-1, 16]               0
            Linear-9                    [-1, 1]              17
Total params: 111,265
Trainable params: 111,265
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.01
Params size (MB): 0.42
Estimated Total Size (MB): 0.43
-------------------------------------------

# Training 

In [ ]:
# # Chnage Y to tensor
# Y = torch.tensor(Y,dtype=torch.float32)
# R_Q_ALL = torch.tensor(r_q_all,dtype=torch.float32)
# # Split the data into training and testing
# Y_train, Y_test,Pos_train,Pos_test = train_test_split(Y,R_Q_ALL, test_size=0.2, random_state=0)

# train_dataset = TensorDataset(Y_train, Pos_train)
# batch_size = 64
# train_loader = DataLoader(train_dataset,batch_size, shuffle=True)

# for batch in train_loader:
#     inputs, targets = batch  # Y, R
#     print(f"The input size: {inputs.size()}")  # Should be [batch_size, 40]
#     print(f"The target size: {targets.size()}")  # Should be [batch_size, 1]
#     break

In [ ]:
Y = torch.tensor(Y, dtype=torch.float32)
R_Q_ALL = torch.tensor(r_q_all, dtype=torch.float32)
Y_train, Y_test, R_train, R_test = train_test_split(Y, R_Q_ALL, test_size=0.2, random_state=0)

# Normalize inputs (Y)
Y_mean, Y_std = Y_train.mean(dim=0), Y_train.std(dim=0)
Y_train = (Y_train - Y_mean) / Y_std
Y_test = (Y_test - Y_mean) / Y_std

# Normalize outputs (R)
R_mean, R_std = R_train.mean(), R_train.std()
R_train = (R_train - R_mean) / R_std
# R_test = (R_test - R_mean) / R_std
print(f"mean of test:")
# Prepare DataLoader
train_dataset = TensorDataset(Y_train, R_train)
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size, shuffle=True)

# Checking the input and target sizes in DataLoader
for batch in train_loader:
    inputs, targets = batch  # Y, R
    print(f"The input size: {inputs.size()}")  # Should be [batch_size, 40]
    print(f"The target size: {targets.size()}")  # Should be [batch_size, 1]
    break

The input size: torch.Size([32, 40])
The target size: torch.Size([32])


In [15]:
print(f"mean of test:{R_test.mean()}, mean of train = {R_mean}")
print(f"std of test:{R_test.std()}, std of train = {R_std}")

criterion = nn.MSELoss()
with torch.no_grad():
    model.eval()
    print(Y_test[2]- Y_test[1])
    print("######")
    print(Y_test[1])
    test_outputs = model(Y_test[2].unsqueeze(0))
    test_outputs = test_outputs * R_std + R_mean
    print(f"Test Output: {test_outputs.item()}")
    print(f"Test Target: {R_test[2].item()}")
    # test_loss = criterion(test_outputs, R_test.view(-1, 1))
    # print(f"Test Loss: {test_loss.item()}")

mean of test:15.833559036254883, mean of train = 15.873400688171387
std of test:6.673295021057129, std of train = 6.676202297210693
tensor([-1.3841, -1.3841, -1.3841, -1.3841, -1.3841, -1.3841, -1.3841, -1.3841,
        -1.3841, -1.3841, -1.3841, -1.3841, -1.3841, -1.3841, -1.3841, -1.3841,
        -1.3841, -1.3841, -1.3841, -1.3841,  0.7282,  0.7282,  0.7282,  0.7282,
         0.7282,  0.7282,  0.7282,  0.7282,  0.7282,  0.7282,  0.7282,  0.7282,
         0.7282,  0.7282,  0.7282,  0.7282,  0.7282,  0.7282,  0.7282,  0.7282])
######
tensor([ 0.1455,  0.1455,  0.1455,  0.1455,  0.1455,  0.1455,  0.1455,  0.1455,
         0.1455,  0.1455,  0.1455,  0.1455,  0.1455,  0.1455,  0.1455,  0.1455,
         0.1455,  0.1455,  0.1455,  0.1455, -1.4164, -1.4164, -1.4164, -1.4164,
        -1.4164, -1.4164, -1.4164, -1.4164, -1.4164, -1.4164, -1.4164, -1.4164,
        -1.4164, -1.4164, -1.4164, -1.4164, -1.4164, -1.4164, -1.4164, -1.4164])
Test Output: 16.050949096679688
Test Target: 4.168258190155

### Loss Function and optimizer 

In [5]:
model = Complex_Model(n_inputs=40)
# Define the loss function
criterion = nn.MSELoss()
# Define the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)

In [6]:
wandb.finish()
num_epochs = 100
wandb.init(project="getting the radius",name = "Normalizing the inputs and outputs + lower learning rate = 0.001 + higher SNR")
wandb.config = {
    "learning_rate": optimizer.param_groups[0]['lr'],
    "epochs": num_epochs,
    "batch_size": train_loader.batch_size,
}

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: mnl03 (mnl03-american-university-of-beirut). Use `wandb login --relogin` to force relogin


### Training Loop 

In [7]:
for epoch in range(num_epochs):
    model.train()
    loss_epoch = 0
    for batch in train_loader:
        inputs, target = batch
        target = target.view(-1, 1)  # Reshape target to [batch_size, 1]
        optimizer.zero_grad()
        # forward pass
        outputs = model(inputs)
        # calculate the loss
        loss = criterion(outputs, target)
        # backward pass
        loss.backward()
        optimizer.step()
        loss_epoch += loss.item()
    avg_loss = loss_epoch / len(train_loader)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss}")
    wandb.log({"epoch": epoch+1, "train_loss": avg_loss})
    if epoch % 10 == 0:
        # test the model
        model.eval()
        with torch.no_grad():
            test_outputs = model(Y_test)
            # Denormalize the outputs
            test_outputs = test_outputs * R_std + R_mean
            test_loss = criterion(test_outputs, R_test.view(-1, 1))  # Reshape Pos_test to [batch_size, 1]
            print(f"Test Loss: {test_loss.item()}")
            scheduler.step(test_loss.item())
            # print the learning  rate
            print(f"Learning rate: {optimizer.param_groups[0]['lr']}")
wandb.finish()

Epoch 1/100, Loss: 1.000434618256986
Test Loss: 44.53860855102539
Learning rate: 0.001
Epoch 2/100, Loss: 1.0001064978018404
Epoch 3/100, Loss: 1.0000613499060274
Epoch 4/100, Loss: 1.0000741175115109
Epoch 5/100, Loss: 1.0000485641136765
Epoch 6/100, Loss: 1.000076632514596
Epoch 7/100, Loss: 1.0001263076364995
Epoch 8/100, Loss: 0.9999939698651433
Epoch 9/100, Loss: 1.0000117888301612
Epoch 10/100, Loss: 1.0000631179064512
Epoch 11/100, Loss: 1.0001053085774183
Test Loss: 44.53230285644531
Learning rate: 0.001
Epoch 12/100, Loss: 1.0001066864803434
Epoch 13/100, Loss: 1.0000598135814072
Epoch 14/100, Loss: 1.0000756141841411
Epoch 15/100, Loss: 1.0000615780502558
Epoch 16/100, Loss: 1.0000830188840628
Epoch 17/100, Loss: 1.00012834122777
Epoch 18/100, Loss: 1.000045526728034
Epoch 19/100, Loss: 1.0000840774476527
Epoch 20/100, Loss: 1.000118600308895
Epoch 21/100, Loss: 1.000051624275744
Test Loss: 44.537376403808594
Learning rate: 0.001
Epoch 22/100, Loss: 1.0000806881859898
Epoch 2

KeyboardInterrupt: 